In [ ]:
##############################
### Install/load packages ###
#############################
library(ggplot2)
library(dplyr)

In [ ]:
#####################
### Load the data ###
#####################
load("country.report.RData")

#for the codes without description, we put the original code
country.report$description <- ifelse( is.na( country.report$Icd10Name), 
                                     country.report$icd_code, 
                                     country.report$Icd10Name)

#remove duplicates mappings for ICD 518.81 that maps to 4 ICD10 J96.0, J96.00, J96.9, J96.90
country.report <- country.report[- which(country.report$siteid == "Italy" & country.report$icd_code %in%c("J96.00", "J96.9", "J96.90"))
, ]

#add the description of one code not added yet U07.2
country.report[ country.report$icd_code == "U07.2", "description"] <- "COVID-19, virus not identified"

In [ ]:
###########################################
### Format the data for ggplot barplot ###
##########################################
selection_df <- country.report %>% 
  group_by(siteid) %>% 
  top_n(5, FractionPatients) %>% 
  group_by(siteid, description) %>% 
  arrange(desc(FractionPatients)) %>%                           
  ungroup() %>%
  mutate(cite_sym = 
           factor(paste(description, siteid, sep = '_'), 
                  levels = rev(paste(description, siteid, sep = '_')))) 

In [ ]:
######################################
### Plot all the graphics together ###
######################################
country_cols <- c(
  'Italy' = '#009E73',
  'France' = '#0072B2',
  'Germany' = '#E69F00',
  'USA' = '#D55E00',
  'Singapore' = '#CC79A7',
  'Combined' = '#444444'
)

ggplot(data = selection_df, 
         aes(x = reorder(cite_sym, FractionPatients), 
             y = FractionPatients)) +
  geom_bar(aes(fill = siteid), stat = 'identity') +
  facet_wrap(vars(siteid), ncol = 1, scales = 'free') +
  coord_flip(ylim = c(0, 1)) +
  theme_bw() +
  theme(legend.position = 'none', strip.placement = 'horizontal') +
  scale_y_continuous(labels = scales::percent_format(), 
                     breaks = seq(0, 1, 0.2),
                     expand = expansion(mult = c(0, 0), add = c(0, 0.02))) +
  scale_x_discrete(labels = function(x) gsub('_.+$', '', x)) +
  labs(title = NULL, x = NULL, y = 'Percentage of patients') +
  scale_fill_manual('legend', values = country_cols)